In [2]:
!pip install stable-baselines3[extra]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.4/178.4 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 42.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 87.8 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=4b5965a340a7f5b44e9d47f0453e3b9b2a768dd9c97c132cdb149918b33558f3
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license


In [3]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

## Load Environment
### using Environment from Gymnasium. [link](https://gymnasium.farama.org/environments/classic_control/cart_pole/)

In [4]:
env = gym.make('CartPole-v1')

In [18]:
##TEST
episodes = 5
for episode in range (episodes):
  state = env.reset()
  finished = False
  score = 0

  while not finished:
    env.render()
    action = env.action_space.sample()
    n_state, reward, finished, info, tu = env.step(action)
    score += reward
  print("Episode:{}, Score:{}".format(episode, score))
env.close()

Episode:0, Score:21.0
Episode:1, Score:23.0
Episode:2, Score:38.0
Episode:3, Score:26.0
Episode:4, Score:27.0


### Understanding the Environment

In [19]:
env.action_space

Discrete(2)

In [22]:
env.action_space.sample()

0

In [20]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [23]:
env.observation_space.sample()

array([ 4.1437817e+00, -6.7317982e+37,  2.3806340e-01, -1.8197745e+38],
      dtype=float32)

### Train the RL model

In [24]:
log_path = os.path.join('Training', 'Log')

In [25]:
log_path

'Training/Log'

In [26]:
env = gym.make('CartPole-v1')
env = DummyVecEnv([lambda : env])
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log= log_path)

Using cpu device


In [29]:
model.learn(total_timesteps= 20000)

Logging to Training/Log/PPO_2
-----------------------------
| time/              |      |
|    fps             | 1803 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1377        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.006309529 |
|    clip_fraction        | 0.0389      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.539      |
|    explained_variance   | 0.954       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.331       |
|    n_updates            | 110         |
|    policy_gradient_loss | -0.00329    |
|    value_loss           | 6.89        |
-----------------------------------------
----

### Save and reload the model

In [30]:
PPO_Path = os.path.join('Traning', 'Save_Models', 'PPO_Model_Cartpole')

In [31]:
model.save(PPO_Path)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:278: UserWarning: Path 'Traning/Save_Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [32]:
# model = PPO.load(PPO_Path, env = env)

### Evaluation

In [33]:
evaluate_policy(model, env, n_eval_episodes=10, render = True)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/base_vec_env.py:234: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(500.0, 0.0)

In [34]:
env.close

<bound method DummyVecEnv.close of <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7fd157df5780>>

### Test the model

In [37]:
##TEST
episodes = 5
for episode in range (episodes):
  obs = env.reset()
  finished = False
  score = 0

  while not finished:
    env.render()
    action, _ = model.predict(obs)
    obs, reward, finished, info = env.step(action) ##
    score += reward
  print("Episode:{}, Score:{}".format(episode, score))
env.close()

Episode:0, Score:[500.]
Episode:1, Score:[500.]
Episode:2, Score:[500.]
Episode:3, Score:[500.]
Episode:4, Score:[500.]


### Viewing Log in Tensorboard

In [40]:
training_log_path = os.path.join(log_path, 'PPO_2')

In [42]:
training_log_path

'Training/Log/PPO_2'

In [43]:
!tensorboard --logdir={training_log_path}

2023-06-30 13:35:59.010848: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required by /usr/local/lib/python3.10/dist-packages/tensorboard_data_server/bin/server)
/usr/local/lib/python3.10/dist-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.34' not found (required by /usr/local/lib/python3.10/dist-packages/tensorboard_data_server/bin/server)
/usr/local/lib/python3.10/dist-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found (required by /usr/local/lib/python3.10/dist-packages/tensorboard_data_server/bin/server)
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.12.3 at http://localhost:6006/ (Press CTRL+C to quit)
Exception ignored in: <module 

### Adding Callback to the training stage

In [44]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [45]:
save_path = os.path.join('Training', 'Saved Models')

In [47]:
from math import e
stop_callback = StopTrainingOnRewardThreshold(reward_threshold= 500, verbose=1)
eval_callback = EvalCallback(env,
                             callback_on_new_best=stop_callback,
                             eval_freq=1000,
                             best_model_save_path= save_path,
                             verbose = 1)

In [48]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log= log_path)

Using cpu device


In [49]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Log/PPO_3
Eval num_timesteps=1000, episode_reward=9.80 +/- 0.40
Episode length: 9.80 +/- 0.40
---------------------------------
| eval/              |          |
|    mean_ep_length  | 9.8      |
|    mean_reward     | 9.8      |
| time/              |          |
|    total_timesteps | 1000     |
---------------------------------


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


New best mean reward!
Eval num_timesteps=2000, episode_reward=10.00 +/- 0.63
Episode length: 10.00 +/- 0.63
---------------------------------
| eval/              |          |
|    mean_ep_length  | 10       |
|    mean_reward     | 10       |
| time/              |          |
|    total_timesteps | 2000     |
---------------------------------
New best mean reward!
-----------------------------
| time/              |      |
|    fps             | 1466 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
Eval num_timesteps=3000, episode_reward=264.60 +/- 144.33
Episode length: 264.60 +/- 144.33
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 265         |
|    mean_reward          | 265         |
| time/                   |             |
|    total_timesteps      | 3000        |
| train/                  |             |
|    approx_kl            | 0.008275

### Using other Algorithms

In [51]:
net_arch = [dict( pi =[128,128,128,128], vf = [128,128,128,128]) ]


In [53]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log= log_path, policy_kwargs={'net_arch':net_arch})

Using cpu device


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/policies.py:460: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [54]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Log/PPO_4
Eval num_timesteps=1000, episode_reward=25.80 +/- 6.58
Episode length: 25.80 +/- 6.58
---------------------------------
| eval/              |          |
|    mean_ep_length  | 25.8     |
|    mean_reward     | 25.8     |
| time/              |          |
|    total_timesteps | 1000     |
---------------------------------


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=2000, episode_reward=27.00 +/- 9.74
Episode length: 27.00 +/- 9.74
---------------------------------
| eval/              |          |
|    mean_ep_length  | 27       |
|    mean_reward     | 27       |
| time/              |          |
|    total_timesteps | 2000     |
---------------------------------
-----------------------------
| time/              |      |
|    fps             | 1105 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
Eval num_timesteps=3000, episode_reward=401.80 +/- 118.20
Episode length: 401.80 +/- 118.20
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 402         |
|    mean_reward          | 402         |
| time/                   |             |
|    total_timesteps      | 3000        |
| train/                  |             |
|    approx_kl            | 0.015345268 |
|    clip_fraction        | 0.221     

### Using DQN

In [55]:
from stable_baselines3 import DQN

In [57]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [58]:
model.learn(total_timesteps= 20000)

Logging to Training/Log/DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.962    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 3182     |
|    time_elapsed     | 0        |
|    total_timesteps  | 79       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.935    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 3759     |
|    time_elapsed     | 0        |
|    total_timesteps  | 136      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.908    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 4386     |
|    time_elapsed     | 0        |
|    total_timesteps  | 194      |
----------------------------------
-------------------------